In [1]:
import numpy as np
import pandas as pd
from scrapy.selector import Selector
from selenium import webdriver 
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
from tqdm import tqdm
import warnings
import re
warnings.filterwarnings("ignore")

In [2]:
# Instantiate the data required in lists
users_list = ['ur117926588', 'ur15298231', 'ur1994077', 'ur17646017', 'ur4532636', 'ur22171966']
user_id = []
movie_title_list = []
year_list = []
review_title_list = []
review_list = []
error_msg = []
review_list2 = []

In [3]:
for user in range(len(users_list)):
    print("Scraping for User {}".format(user+1))
#     driver = webdriver.Chrome('chromedriver.exe')
    driver = webdriver.Chrome(ChromeDriverManager().install())
    url = 'https://www.imdb.com/user/{}/reviews'.format(users_list[user])
    time.sleep(1)
    driver.get(url)
    time.sleep(1)
    
    sel = Selector(text = driver.page_source)
    num_of_reviews = sel.css(".header span::text").extract_first().replace(',','').split(' ')[0]
    more_review_pages = int(int(num_of_reviews)/25)
    
    user_id += [users_list[user] for i in range(int(num_of_reviews))]
    
    # Loading all the reviews in the single page before scraping
    for i in tqdm(range(more_review_pages)):
        try:
            css_selector = 'load-more-trigger'
            driver.find_element(By.ID, css_selector).click()
            time.sleep(2)
        except:
            pass
    
    reviews = driver.find_elements(By.CSS_SELECTOR, 'div.review-container')
    for r in tqdm(reviews):
        try:
            sel2 = Selector(text = r.get_attribute('innerHTML'))
            try:
                movie_title = sel2.css('.lister-item-header a::text').extract_first().strip()
            except:
                movie_title = np.NaN
            try:
                year = sel2.css('.lister-item-year.text-muted.unbold::text').extract_first().strip().replace('(','').replace(')','')
                year = re.sub(r'[a-zA-Z\s]+', '', year)
            except:
                year = np.NaN
            try:
                review_title = sel2.css('.title::text').extract_first().strip()
            except:
                review_title = np.NaN
            try:
                # extract all paragraphs inside review
                review = sel2.css('.text.show-more__control::text').extract()
                review = ' '.join(review)
            except:
                review = np.NaN
                
            movie_title_list.append(movie_title)
            year_list.append(year)
            review_title_list.append(review_title)
            review_list.append(review)
        except Exception as e:
            error_msg.append(e)

Scraping for User 1


100%|██████████████████████████████████████████████████████████████████████████████| 1254/1254 [00:25<00:00, 50.14it/s]


Scraping for User 2


100%|██████████████████████████████████████████████████████████████████████████████| 2186/2186 [00:53<00:00, 40.84it/s]


Scraping for User 3


...

100%|██████████████████████████████████████████████████████████████████████████████| 1753/1753 [00:36<00:00, 47.59it/s]


Scraping for User 5


100%|██████████████████████████████████████████████████████████████████████████████| 3103/3103 [00:50<00:00, 61.79it/s]


Scraping for User 6


100%|██████████████████████████████████████████████████████████████████████████████| 2010/2010 [00:30<00:00, 66.37it/s]


In [6]:
# Storing all data in dataframe
reviews_df = pd.DataFrame({
    "UserID": user_id,
    "Title": movie_title_list,
    "Year": year_list,
    "Review_Title": review_title_list,
    "Review": review_list})

In [7]:
# Export dataset
reviews_df.to_csv(path_or_buf = "users_reviews.csv"
                          , index = False, encoding='utf-8-sig')